<a href="https://colab.research.google.com/github/sittidetw/dads5001_midterm_project/blob/main/DADS5001_Midterm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sittidetw/dads5001_midterm_project.git

fatal: destination path 'dads5001_midterm_project' already exists and is not an empty directory.


In [2]:
pip install python-calamine

In [40]:
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import gc

# Data Cleaning

In [41]:
# HIV testing and know result by key populations
path = "/content/dads5001_midterm_project/rd_testing_population/*.xlsx"

files = glob.glob(path)
print("Total files:", len(files))
df_test_pop_combine = None
for file in files:
  print(file)

  # Load the Excel file
  df = pd.read_excel(file, engine='calamine', header=None, skiprows=2, skipfooter=1)
  file_name = Path(file).name
  file_period = file_name.split('_')
  period = file_period[2]

  df = df.transpose()
  df.iloc[:,0:3] = df.iloc[:,0:3].ffill()
  #df.insert(loc=0, column='header', value=df.index)
  #df

  # Define conditions
  conditions = [
      (df[0] == 'จังหวัด'),
      (df[0] == 'เขตสุขภาพ'),
      (df[0].str.contains('testing')) & (df[1].str.contains('Female')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Male')),
      (df[0].str.contains('testing')) & (df[1].str.contains('All KP')),
      (df[0].str.contains('testing')) & (df[1].str.contains('Total')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Female')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Male')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('All KP')),
      (df[0].str.contains('Positive')) & (df[1].str.contains('Total'))
  ]

  # Define the corresponding values for each condition
  values = ['province', 'health_region',
            'test_female', 'test_male', 'test_all_kp', 'test_total',
            'positive_female', 'positive_male', 'positive_all_kp', 'positive_total'
  ]

  # Create the new column
  df['header_temp'] = np.select(conditions, values, default='Unknown') # default is for cases that don't match any condition
  df['header'] = df['header_temp'].str.cat(df[2].str.replace(' ', '_').str.lower(), sep='_').fillna(df['header_temp'])
  df['header'] = df['header'].str.replace('total_total','grand_total')
  columns_to_drop = [0,1,2,'header_temp']
  df = df.drop(columns=columns_to_drop)
  df = df.transpose()
  df.columns = df.loc['header'].values
  df = df.drop(index='header')

  # Add Period
  df.insert(loc=0, column='quarter', value=period)
  df.insert(loc=0, column='q', value=period[-2:])
  df.insert(loc=0, column='year', value=period[:4])

  # Change data type in dataframe
  for col in df.iloc[0:,5:].columns: df[col] = df[col].astype('int32')
  for col in df.iloc[0:,0:5].columns: df[col] = df[col].astype('string')

  #df
  if (df_test_pop_combine is None):
    df_test_pop_combine = df
  else:
    df_test_pop_combine = pd.concat([df_test_pop_combine, df], ignore_index=True)

display(df_test_pop_combine.groupby(['year','quarter'])['positive_grand_total'].count())


Total files: 7
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2024Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2023Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2023Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2025Q4_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2024Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2026Q1_20260215.xlsx
/content/dads5001_midterm_project/rd_testing_population/Testing_Population_2025Q1_20260215.xlsx


year  quarter
2023  2023Q1     77
      2023Q4     77
2024  2024Q1     77
      2024Q4     77
2025  2025Q1     77
      2025Q4     77
2026  2026Q1     77
Name: positive_grand_total, dtype: int64

In [42]:
# Transform accumulated data to total year summary
df_test_pop = df_test_pop_combine[df_test_pop_combine['q'] == 'Q4'].copy()
df_test_pop_temp1 = df_test_pop_combine[(df_test_pop_combine['q'] == 'Q1') & (df_test_pop_combine['year'].isin(df_test_pop['year'].unique()))].copy()

next_years_int = df_test_pop['year'].astype(int).unique() + 1
next_years_str = next_years_int.astype(str)
df_test_pop_temp2 = df_test_pop_combine[(df_test_pop_combine['q'] == 'Q1') & (df_test_pop_combine['year'].isin(next_years_str))].copy()

df_test_pop = df_test_pop.set_index(['year', 'province'])
df_test_pop_temp1 = df_test_pop_temp1.set_index(['year', 'province'])

df_test_pop_temp2['year'] = df_test_pop_temp2['year'].astype(int) - 1
df_test_pop_temp2['year'] = df_test_pop_temp2['year'].astype(str)
df_test_pop_temp2 = df_test_pop_temp2.set_index(['year', 'province'])

cols_to_cal = df_test_pop.columns[3:]
df_test_pop[cols_to_cal] = (df_test_pop[cols_to_cal] - df_test_pop_temp1[cols_to_cal]) + df_test_pop_temp2[cols_to_cal]

df_test_pop = df_test_pop.reset_index(drop=False)
df_test_pop = df_test_pop.drop(columns=['q','quarter'])
df_test_pop.iloc[:, 3:] = df_test_pop.iloc[:, 3:].clip(lower=0)

# Delete temp variables
del df_test_pop_temp1
del df_test_pop_temp2
gc.collect()

test_male = [col for col in df_test_pop.columns if col.startswith('test_male_') and 'all_kps' not in col and 'total' not in col]
test_female = [col for col in df_test_pop.columns if col.startswith('test_female_') and 'all_kps' not in col and 'total' not in col]
positive_male = [col for col in df_test_pop.columns if col.startswith('positive_male_') and 'all_kps' not in col and 'total' not in col]
positive_female = [col for col in df_test_pop.columns if col.startswith('positive_female_') and 'all_kps' not in col and 'total' not in col]

df_test_pop['test_male_all_kps'] = df_test_pop[test_male].sum(axis=1)
df_test_pop['test_female_all_kps'] = df_test_pop[test_female].sum(axis=1)
df_test_pop['positive_male_all_kps'] = df_test_pop[positive_male].sum(axis=1)
df_test_pop['positive_female_all_kps'] = df_test_pop[positive_female].sum(axis=1)

df_test_pop['test_all_kp_total'] = df_test_pop[['test_male_all_kps','test_female_all_kps']].sum(axis=1)
df_test_pop['positive_all_kp_total'] = df_test_pop[['positive_male_all_kps','positive_female_all_kps']].sum(axis=1)

# Add sum of others
added_col = ['test_male_others','test_female_others','test_all_others_total','positive_male_others','positive_female_others','positive_all_others_total']
col_name = ['test_male_all_kps','test_female_all_kps','test_all_kp_total','positive_male_all_kps','positive_female_all_kps','positive_all_kp_total']
col_ttl = ['test_male_total','test_female_total','test_grand_total','positive_male_total','positive_female_total','positive_grand_total']
indices = df_test_pop.columns.get_indexer(col_name)
for i in reversed(range(len(indices))):
    indice = indices[i]
    new_col = added_col[i]
    source_ttl = col_ttl[i]
    source_kp = col_name[i]
    df_test_pop.insert(loc=indice + 1, column=new_col, value=df_test_pop[source_ttl] - df_test_pop[source_kp])

display(df_test_pop)
df_test_pop.to_excel('test_pop.xlsx', index=False)

,year,province,health_region,test_male_fsw,test_male_migrant,test_male_msm,test_male_msw,test_male_partner_of_hiv,test_male_prisoner,test_male_pwid,...,positive_female_prisoner,positive_female_pwid,positive_female_tg,positive_female_tgsw,positive_female_all_kps,positive_female_others,positive_female_total,positive_all_kp_total,positive_all_others_total,positive_grand_total
0,2024,เชียงใหม่,1,0,2,19081,796,72,240,218,...,4,2,1,0,30,139,169,335,381,716
1,2024,ลำพูน,1,0,0,3174,0,8,3,0,...,1,0,0,0,4,26,30,29,64,93
2,2024,ลำปาง,1,0,0,2721,7,18,7,2,...,0,0,0,0,8,35,43,77,87,164
3,2024,แพร่,1,0,0,818,4,1,0,0,...,0,0,0,0,0,26,26,19,95,114
4,2024,น่าน,1,0,0,1539,2,18,185,2,...,0,0,0,0,1,13,14,41,43,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2025,พัทลุง,12,0,1,2508,69,19,1870,83,...,1,1,0,0,17,24,41,55,46,101
227,2025,ปัตตานี,12,0,0,518,1,3,2568,107,...,0,1,0,0,1,8,9,29,27,56
228,2025,ยะลา,12,0,0,100,0,4,2090,2066,...,0,0,0,0,0,12,12,41,21,62
229,2025,นราธิวาส,12,0,0,261,6,2,219,479,...,0,0,1,0,3,18,21,26,46,72


In [43]:
df_test_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231 entries, 0 to 230
Data columns (total 57 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   year                            231 non-null    string
 1   province                        231 non-null    string
 2   health_region                   231 non-null    string
 3   test_male_fsw                   231 non-null    int32 
 4   test_male_migrant               231 non-null    int32 
 5   test_male_msm                   231 non-null    int32 
 6   test_male_msw                   231 non-null    int32 
 7   test_male_partner_of_hiv        231 non-null    int32 
 8   test_male_prisoner              231 non-null    int32 
 9   test_male_pwid                  231 non-null    int32 
 10  test_male_tg                    231 non-null    int32 
 11  test_male_tgsw                  231 non-null    int32 
 12  test_male_all_kps               231 non-null    in